<a href="https://colab.research.google.com/github/hansong0219/Advanced_Deep_Learning/blob/master/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>